In [1]:
import pandas as pd
import numpy as np
import xlrd
import os
from datetime import datetime, date
from time import mktime
import time

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
# Import Raw Data from excel files into dataFrames
df10=pd.read_excel('orig_data/mlb_odds_2010.xlsx')
df10['Year'] = 2010
df11=pd.read_excel('orig_data/mlb_odds_2011.xlsx')
df11['Year'] = 2011
df12=pd.read_excel('orig_data/mlb_odds_2012.xlsx')
df12['Year'] = 2012
df13=pd.read_excel('orig_data/mlb_odds_2013.xlsx')
df13['Year'] = 2013
df14=pd.read_excel('orig_data/mlb_odds_2014.xlsx')
df14['Year'] = 2014
df15=pd.read_excel('orig_data/mlb_odds_2015.xlsx')
df15['Year'] = 2015
df16=pd.read_excel('orig_data/mlb_odds_2016.xlsx')
df16['Year'] = 2016
df17=pd.read_excel('orig_data/mlb_odds_2017.xlsx')
df17['Year'] = 2017
df18=pd.read_excel('orig_data/mlb_odds_2018.xlsx')
df18['Year'] = 2018
df19=pd.read_excel('orig_data/mlb_odds_2019.xlsx')
df19['Year'] = 2019

In [4]:
# make list of dataFrames
dfList = [df10, df11, df12, df13, df14, df15, df16, df17, df18, df19]

In [5]:
# reformat data
for df in dfList:
    try:
        dfnew=df['RL'].str.split(pat="(", n=1, expand=True)
        dfnewer = dfnew[1].str.rstrip("\)")
        df['run_line_close'] = dfnew[0].apply(float)
        df['run_line_odds_close'] = dfnewer.apply(int)
    except KeyError:
        pass

In [6]:
for df in dfList:
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df.rename({"vh": "visitor_home", "final": "team_run_total", "close":"money_line_close", "open":"money_line_open", "openou":"over_under_line_open", "open_ou":"over_under_line_open", "unnamed:_18":"over_under_odds_open", "unnamed:_19":"over_under_odds_open", "close_ou":"over_under_line_close", "closeou":"over_under_line_close", "unnamed:_20":"over_under_odds_close", "unnamed:_21":"over_under_odds_close"}, axis='columns', inplace=True)    

In [7]:
def convt(num):
    if type(num) == str:
        num = float(num[0]) + .5
    return num
for df in dfList:
    df['over_under_line_close'] = df['over_under_line_close'].apply(convt)

In [8]:
# create additional columns
for df in dfList:
    totRuns = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.team_run_total == "NL":
                totRuns.append(np.nan)
            elif index % 2 == 0:
                counter = 0
                counter += row.team_run_total
            else:
                counter += row.team_run_total
                totRuns.append(counter)
                totRuns.append(counter)
    except TypeError:
        print(row)
    df['total_runs_game'] = totRuns

In [9]:
for df in dfList:
    RunDif = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.team_run_total == "NL":
                RunDif.append(np.nan)
            elif index % 2 == 0:
                counter = 0
                team1 = row.team_run_total
            else:
                team2 = row.team_run_total
                dif = team1 - team2
                RunDif.append(dif)
                RunDif.append(-dif)
    except TypeError:
        print(row)
    df['run_dif_game'] = RunDif

In [10]:
for df in dfList:
    df['date'] = df['year'].astype(str) + '-' + df['date'].astype(str).apply(lambda x: '0' + x if len(x) == 3 else x)
    df['date'] = df['date'].apply(lambda x: x[:7] + '-' + x[7:])

In [11]:
df1 = pd.concat(dfList, sort=True, ignore_index=True)

In [12]:
for dt in df1['date']:
    yr = int(dt[0:4])
    mon = int(dt[5:7])
    day = int(dt[8:10])
    
    dt2 = datetime(yr,mon,day)
    unix = mktime(dt2.timetuple())
    df['date'] = unix

In [13]:
df1.drop(['year', 'rl', '1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th'], axis=1, inplace=True)

In [14]:
df1 = df1[df1['visitor_home'] != 'N']

In [15]:
colNew = []
i = 0
for index, row in df1.iterrows():
    if index == 0:
        pass
    elif index % 2 == 0:
        i += 1
    colNew.append(i)
df1['gameno'] = colNew
df1.replace('LOS','LAD',inplace=True)
df1.replace('CUB','CHC',inplace=True)
df1.replace('SDG','SD',inplace=True)
df1.replace('SFO','SF',inplace=True)
df1.replace('KAN','KC',inplace=True)
df1.replace('TAM','TB',inplace=True)

In [16]:
df1 = df1.reindex(columns=['date', 'gameno', 'team', 'visitor_home', 'team_run_total', 'total_runs_game', 'money_line_close', 'money_line_open', 'over_under_line_close', 'over_under_line_open', 'over_under_odds_close', 'over_under_odds_open', 'pitcher', 'rot', 'run_dif_game', 'run_line_close', 'run_line_odds_close'])

In [17]:
#df1.replace(['NL', 'nan'], np.nan, inplace=True)

In [18]:
#merge visitor and home rows together
df_v = df1.loc[df1['visitor_home'] == 'V']
df_h = df1.loc[df1['visitor_home'] == 'H']
df_h.columns = ['h_date', 'gameno', 'h_team', 'home', 'h_team_run_total',
                   'h_total_runs_game', 'h_money_line_close', 'h_money_line_open',
                   'under_line_close', 'under_line_open',
                   'under_odds_close', 'under_odds_open', 'h_pitcher', 'h_rot',
                   'h_run_dif_game', 'h_run_line_close', 'h_run_line_odds_close']
df_v.columns = ['date', 'gameno', 'v_team', 'visitor', 'v_team_run_total',
                   'total_runs_game', 'v_money_line_close', 'v_money_line_open',
                   'over_line_close', 'over_line_open',
                   'over_odds_close', 'over_odds_open', 'v_pitcher', 'v_rot',
                   'v_run_dif_game', 'v_run_line_close', 'v_run_line_odds_close']

df3 = pd.merge(df_v, df_h, on='gameno', how='right')

In [19]:
#drop unnecessary columns
df3.drop(['h_total_runs_game','h_date'], axis=1, inplace=True)

In [20]:
#reindex columns
df_final = df3.reindex(columns = ['date', 'gameno', 
             
             'visitor', 'v_team', 'v_team_run_total', 
             'v_money_line_close', 'v_money_line_open', 
             'v_run_line_close', 'v_run_line_odds_close', 
             'v_pitcher', 'v_rot', 'v_run_dif_game',
             
             'home', 'h_team', 'h_team_run_total', 
             'h_money_line_close', 'h_money_line_open', 
             'h_run_line_close', 'h_run_line_odds_close',
             'h_pitcher', 'h_rot', 'h_run_dif_game',
             
             'total_runs_game', 
             'over_line_close', 'over_line_open', 'over_odds_close', 'over_odds_open', 
             'under_line_close', 'under_line_open', 'under_odds_close', 'under_odds_open' 
            ])

In [21]:
df_final.replace(np.nan, 0)
df_final

,date,gameno,visitor,v_team,v_team_run_total,v_money_line_close,v_money_line_open,v_run_line_close,v_run_line_odds_close,v_pitcher,v_rot,v_run_dif_game,home,h_team,h_team_run_total,h_money_line_close,h_money_line_open,h_run_line_close,h_run_line_odds_close,h_pitcher,h_rot,h_run_dif_game,total_runs_game,over_line_close,over_line_open,over_odds_close,over_odds_open,under_line_close,under_line_open,under_odds_close,under_odds_open
0,2010-04-04,0,V,NYY,7,107.0,104,NaN,NaN,CSABATHIA-L,931,-2.0,H,BOS,9,-117.0,-114,NaN,NaN,JBECKETT-R,932,2.0,16.0,9.0,9.0,-105,-116,9.0,9.0,-115,-104
1,2010-04-05,1,V,PHI,11,-200.0,-200,NaN,NaN,RHALLADAY-R,901,10.0,H,WAS,1,175.0,175,NaN,NaN,JLANNAN-L,902,-10.0,12.0,7.5,7.5,-125,100,7.5,7.5,105,-120
2,2010-04-05,2,V,MIA,1,-102.0,105,NaN,NaN,JJOHNSON-R,903,-6.0,H,NYM,7,-108.0,-115,NaN,NaN,JOSANTANA-L,904,6.0,8.0,7.0,7.0,-130,-125,7.0,7.0,110,105
3,2010-04-05,3,V,STL,11,-143.0,-155,NaN,NaN,CARPENTER-R,905,5.0,H,CIN,6,128.0,140,NaN,NaN,AHARANG-R,906,-5.0,17.0,7.5,7.5,-105,-105,7.5,7.5,-115,-115
4,2010-04-05,4,V,LAD,5,-126.0,-155,NaN,NaN,VPADILLA-R,907,-6.0,H,PIT,11,111.0,140,NaN,NaN,ZDUKE-L,908,6.0,16.0,8.5,8.5,-120,-110,8.5,8.5,100,-110
5,2010-04-05,5,V,COL,5,-105.0,110,NaN,NaN,UJIMENEZ-R,909,2.0,H,MIL,3,-105.0,-125,NaN,NaN,YGALLARDO-R,910,-2.0,8.0,7.5,7.5,105,-105,7.5,7.5,-125,-115
6,2010-04-05,6,V,CHC,5,110.0,115,NaN,NaN,CZAMBRANO-R,911,-11.0,H,ATL,16,-125.0,-130,NaN,NaN,DLOWE-R,912,11.0,21.0,8.5,8.5,100,-110,8.5,8.5,-120,-110
7,2010-04-05,7,V,SD,3,166.0,155,NaN,NaN,JGARLAND-R,913,-3.0,H,ARI,6,-186.0,-175,NaN,NaN,DHAREN-R,914,3.0,9.0,8.0,8.0,-120,-110,8.0,8.0,100,-110
8,2010-04-05,8,V,SF,5,-147.0,-130,NaN,NaN,TLINCECUM-R,915,3.0,H,HOU,2,132.0,115,NaN,NaN,ROSWALT-R,916,-3.0,7.0,7.0,7.5,-110,105,7.0,7.5,-110,-125
9,2010-04-05,9,V,CLE,0,127.0,150,NaN,NaN,WESTBROOK-R,917,-6.0,H,CWS,6,-142.0,-170,NaN,NaN,MBUEHRLE-L,918,6.0,6.0,9.0,9.0,-105,-105,9.0,9.0,-115,-115


In [22]:
pickle_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), 'data', 'dataset.pickle'))
df_final.to_pickle(pickle_path)

In [23]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22326 entries, 0 to 22325
Data columns (total 31 columns):
date                     22326 non-null object
gameno                   22326 non-null int64
visitor                  22326 non-null object
v_team                   22326 non-null object
v_team_run_total         22326 non-null object
v_money_line_close       22326 non-null float64
v_money_line_open        22326 non-null int64
v_run_line_close         12461 non-null float64
v_run_line_odds_close    12461 non-null float64
v_pitcher                22306 non-null object
v_rot                    22326 non-null int64
v_run_dif_game           22325 non-null float64
home                     22326 non-null object
h_team                   22326 non-null object
h_team_run_total         22326 non-null object
h_money_line_close       22326 non-null float64
h_money_line_open        22326 non-null int64
h_run_line_close         12461 non-null float64
h_run_line_odds_close    12461 non-null flo